# Week 2 — Pandas Data Manipulation & ETL

**Course:** Applied ML Foundations for SaaS Analytics  
**Week Focus:** Master Pandas for data cleaning, aggregation, joining multiple datasets, and preparing clean data for analysis.

---

## 🎯 Learning Objectives

By the end of this week, you will:
- Load and inspect data from multiple file formats (CSV, JSON)
- Clean and handle missing data effectively
- Group, aggregate, and pivot data for SaaS analytics
- Join datasets to create comprehensive views
- Detect and handle outliers and anomalies
- Build reliable ETL pipelines

## 📊 Real-World Context

SaaS analytics requires combining multiple data sources:
- **Subscriptions data**: signup dates, churn dates, plan tier, pricing
- **User events**: activity logs, timestamps, event types
- **Feature usage**: which features users interact with
- **Feedback**: customer sentiment, support tickets

Your job: Transform messy, fragmented raw data into clean, joined datasets ready for analysis and modeling.

In [ ]:
from IPython.display import HTML
HTML('''
<style>
details {
  margin: 10px 0;
  padding: 8px 12px;
  border: 1px solid #d9e2ec;
  border-radius: 8px;
  background: #f9fbfd;
}
details summary {
  font-weight: 600;
  color: #0056b3;
  cursor: pointer;
}
details[open] {
  background: #f1f7ff;
  border-color: #c3d4f0;
}
details pre {
  background: #f8f9fa;
  padding: 8px;
  border-radius: 6px;
}
</style>
''')

## 🏢 Scenario — Merging CloudWave Data

Your company stores data in 4 separate systems:
1. **Subscriptions DB**: customer IDs, signup dates, churn dates, plan types
2. **Event streaming logs**: user_id, timestamp, event_type, value
3. **Feature DB**: feature_id, feature_name, category, launch_date
4. **Customer feedback**: user_id, timestamp, category, sentiment, text

**Task**: Create a unified "customer 360" view combining all sources to answer:
- Which customers are at risk of churning?
- Which features drive the most engagement?
- How does feature adoption correlate with retention?

<details>
<summary>💡 Hint — Structured Approach to Data Integration</summary>

**Step 1: Understand the data**
- Load each dataset separately first
- Check shapes, columns, data types
- Look for key identifiers (user_id, feature_id, etc.)

**Step 2: Data quality check**
```python
df.info()  # types and nulls
df.describe()  # numeric summaries
df[df.isnull().any(axis=1)]  # find rows with ANY null
```

**Step 3: Choose join strategy**
- `inner`: only matching rows (stricter, smaller dataset)
- `left`: keep all from left table, add from right
- `outer`: keep all rows, fill NaNs where no match

**Step 4: Validate results**
- Check row count before/after
- Verify no unexpected NaNs
- Spot-check a few rows manually

</details>

<details>
<summary>✅ Solution — Subscription Lifecycle Analysis</summary>

```python
import pandas as pd
import numpy as np
from datetime import datetime

# Load subscriptions
subs = pd.read_csv('../data/subscriptions.csv', parse_dates=['signup_date','churn_date'])

print(f"Total subscriptions: {len(subs)}")
print(f"Churned: {subs['churn_date'].notna().sum()}")
print(f"Active: {subs['churn_date'].isna().sum()}")

# Create useful derived columns
subs['is_churned'] = subs['churn_date'].notna()

# Lifetime: days from signup to churn (or today if active)
today = pd.Timestamp.now()
subs['churn_or_today'] = subs['churn_date'].fillna(today)
subs['lifetime_days'] = (subs['churn_or_today'] - subs['signup_date']).dt.days

# Cohort by signup month
subs['signup_month'] = subs['signup_date'].dt.to_period('M')

# Churn rate by plan
churn_by_plan = subs.groupby('plan_tier').agg({
    'user_id': 'count',
    'is_churned': 'sum'
})
churn_by_plan['churn_rate'] = churn_by_plan['is_churned'] / churn_by_plan['user_id']
print("\nChurn Rate by Plan:")
print(churn_by_plan)

# Average lifetime by cohort
lifetime_by_cohort = subs.groupby('signup_month')['lifetime_days'].mean()
print("\nAverage Lifetime (days) by Signup Cohort:")
print(lifetime_by_cohort)
```

**Key takeaways:**
- Handle dates properly with `pd.to_datetime()`
- Fill NaN values strategically for analysis
- Groupby + agg is the engine for SaaS metrics
- Create cohort columns for temporal analysis

</details>

## 📚 Key Concepts — Pandas Fundamentals

### DataFrames: The Core Abstraction
```
DataFrames = SQL tables + NumPy speed + Flexible labels
```

### Key Operations
1. **Load**: `pd.read_csv()`, `pd.read_json()`, `pd.read_sql()`
2. **Inspect**: `.shape`, `.dtypes`, `.info()`, `.describe()`, `.head()`
3. **Clean**: `.dropna()`, `.fillna()`, `.astype()`
4. **Group**: `.groupby()` → powerful aggregations
5. **Join**: `.merge()` → combine datasets on keys
6. **Pivot**: `.pivot_table()` → reshape data for different views

### Critical Patterns for SaaS

```python
# Find churn cohort
df[df['churn_date'].notna()].groupby('signup_month')['user_id'].count()

# Active users each day
df[df['event_date'] >= '2024-01-01'].groupby(['event_date','user_id']).size()

# Feature adoption by segment
df.merge(feature_df, on='feature_id').groupby(['segment','feature_name'])['usage_count'].sum()
```

## ✍️ Hands-on Exercises

### Exercise 1: Data Cleaning & Preparation
1. Load subscriptions.csv
2. Check for missing values and data types
3. Convert signup_date and churn_date to datetime
4. Create a "is_churned" binary column
5. Create a "lifetime_days" metric (days from signup to churn or today)
6. Handle any edge cases (negative lifetimes, invalid dates)

### Exercise 2: Aggregation & Grouping
1. Load user_events.csv and subscriptions.csv
2. Join them on user_id
3. For each user, compute: total events, first event date, last event date, event frequency
4. Group by plan_tier: which tier has the highest event frequency?
5. Create a cohort analysis: signup_month vs. retention at 30/60/90 days

### Exercise 3: Multi-table Integration
1. Load all 5 datasets
2. Create a user-centric joined table with:
   - Subscription info (plan_tier, signup_date, churn_date)
   - Engagement metrics (total_events, total_feature_usage, feature_count)
   - Feedback sentiment (avg_sentiment, feedback_count)
3. Identify high-engagement, high-satisfaction customers at churn risk

<details>
<summary>💡 Hint — Joining Multiple Tables Effectively</summary>

**Join types visualization:**
```
LEFT (subscription.user_id):  [1, 2, 3, 4, 5]
RIGHT (events.user_id):       [2, 3, 4, 5, 6]

INNER:  [2, 3, 4, 5]  ← only matching
LEFT:   [1, 2, 3, 4, 5]  ← all from left, NaNs on right where no match
OUTER:  [1, 2, 3, 4, 5, 6]  ← all from both
```

**Common pitfall:** Don't `.groupby()` after a join without checking cardinality!
- If events table has multiple rows per user, you'll get multiplied results
- Always aggregate at the appropriate level before joining

</details>

<details>
<summary>✅ Solution — Feature Adoption by Customer Segment</summary>

```python
import pandas as pd
import numpy as np

# Load datasets
subs = pd.read_csv('../data/subscriptions.csv', parse_dates=['signup_date','churn_date'])
feature_usage = pd.read_csv('../data/feature_usage.csv')
feature_cat = pd.read_csv('../data/product_catalog.csv')

# Pre-aggregate: total usage per user per feature (avoid data explosion on join)
user_feature_summary = (feature_usage
    .groupby(['user_id','feature_name'])['usage_count']
    .sum()
    .reset_index()
    .rename(columns={'usage_count':'total_usage'})
)

# Count features per user
features_per_user = (user_feature_summary
    .groupby('user_id')['feature_name']
    .nunique()
    .reset_index()
    .rename(columns={'feature_name':'feature_count'})
)

# Join to subscriptions
result = subs.merge(features_per_user, on='user_id', how='left')
result['feature_count'] = result['feature_count'].fillna(0)

# Analyze by plan tier
adoption_by_plan = result.groupby('plan_tier').agg({
    'feature_count': ['mean', 'median', 'std'],
    'is_churned': 'mean'  # churn rate
}).round(2)

print("Feature adoption by plan tier:")
print(adoption_by_plan)

# Do churned users have lower feature adoption?
active = result[~result['is_churned']]
churned = result[result['is_churned']]
print(f"\nActive users avg features: {active['feature_count'].mean():.2f}")
print(f"Churned users avg features: {churned['feature_count'].mean():.2f}")
```

**Why this approach:**
- Aggregates first to reduce join cardinality
- Fills NaNs strategically (0 for no features = inactive)
- Compares segments (active vs churned) to find patterns

</details>

In [ ]:
# Demo: Pandas ETL pipeline for SaaS data
import pandas as pd
import numpy as np

print("=" * 70)
print("WEEK 2: PANDAS DATA INTEGRATION DEMO")
print("=" * 70)

# Load subscriptions
subs = pd.read_csv('../data/subscriptions.csv', parse_dates=['signup_date', 'churn_date'])
print(f"\n1. SUBSCRIPTIONS DATA")
print(f"   Records: {len(subs):,}")
print(f"   Columns: {', '.join(subs.columns.tolist())}")
print(f"   Churned: {subs['churn_date'].notna().sum():,} ({subs['churn_date'].notna().sum()/len(subs)*100:.1f}%)")

# Load events
events = pd.read_csv('../data/user_events.csv')
print(f"\n2. USER EVENTS DATA")
print(f"   Records: {len(events):,}")
print(f"   Unique users: {events['user_id'].nunique():,}")
print(f"   Columns: {', '.join(events.columns.tolist())}")

# Load feature usage
feature_usage = pd.read_csv('../data/feature_usage.csv')
print(f"\n3. FEATURE USAGE DATA")
print(f"   Records: {len(feature_usage):,}")
print(f"   Unique features: {feature_usage['feature_name'].nunique()}")
print(f"   Columns: {', '.join(feature_usage.columns.tolist())}")

# Create merged view
print(f"\n4. DATA INTEGRATION")

# Aggregate user-level metrics
user_stats = events.groupby('user_id').agg({
    'event_type': 'count'
}).rename(columns={'event_type':'total_events'}).reset_index()

feature_stats = feature_usage.groupby('user_id').agg({
    'usage_count': 'sum',
    'feature_name': 'nunique'
}).rename(columns={'usage_count':'total_usage', 'feature_name':'features_used'}).reset_index()

# Join to subscriptions
merged = subs.merge(user_stats, on='user_id', how='left')
merged = merged.merge(feature_stats, on='user_id', how='left')

print(f"   Merged dataset: {len(merged):,} users")
print(f"   Columns: {merged.shape[1]}")

# Analysis
print(f"\n5. INSIGHTS FROM JOINED DATA")
merged['is_churned'] = merged['churn_date'].notna()
print(f"   Active users with data: {len(merged[merged['is_churned'] == False]):,}")
print(f"   Churned users: {merged['is_churned'].sum():,}")

active = merged[~merged['is_churned']]
churned = merged[merged['is_churned']]

print(f"\n   Active users stats:")
print(f"      Avg events: {active['total_events'].mean():.0f}")
print(f"      Avg features used: {active['features_used'].mean():.1f}")
print(f"\n   Churned users stats:")
print(f"      Avg events: {churned['total_events'].mean():.0f}")
print(f"      Avg features used: {churned['features_used'].mean():.1f}")
print(f"\n   Insight: Churned users have lower feature adoption!")
print("=" * 70)

## 🤔 Reflection & Application

**Question 1:** When should you use `.merge()` vs `.join()`?
- `merge()`: explicit, works on any columns (not just index)
- `join()`: default on index, slightly faster for index-based joins

**Question 2:** What are the risks of `.merge()` without aggregating first?
- Cardinality explosion: 10 subscriptions × 1000 events per user = 10,000 rows!
- Solution: Aggregate to 1 row per user before joining

**Question 3:** How do you validate a join was successful?
- Check row count: should match the "left" table unless duplicates
- Check NaN distribution: if many unexpected NaNs, join keys misaligned
- Spot-check: manually verify a few rows

## 📝 Practice Assignment

**Problem:** Create a customer risk score combining:
1. Churn likelihood (cohort churn rate)
2. Low engagement (below median feature adoption)
3. Declining trend (compare recent usage to historical average)

**Steps:**
1. Load all datasets
2. Create user-level aggregations for each dimension
3. Merge into a unified customer table
4. Compute a composite risk score (0-100)
5. Segment users: low/medium/high risk

**Deliverable:** Jupyter notebook showing the ETL process and final risk segment distribution.

## 🔗 Next Steps

In Week 3, we'll visualize these joined datasets to uncover patterns and tell compelling stories about customer behavior.